In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math

In [40]:
df = pd.read_csv('Data laboratorio 1/students.csv', sep=';')
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [41]:
df['Target'].unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)

In [42]:
#Unificando Graduate + Enrolled
df['Target']=df['Target'].replace('Graduate','Enrolled')
df['Target'].unique()

array(['Dropout', 'Enrolled'], dtype=object)

In [43]:
#Tipos de datos, buscando los datos continuos
df.dtypes

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

In [8]:
print(df.select_dtypes(include=['float']).columns)

Index(['Previous qualification (grade)', 'Admission grade',
       'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)',
       'Unemployment rate', 'Inflation rate', 'GDP'],
      dtype='object')


#### Preprocesing
* Unificar los valores graduate enrolled de Target
* Pasar los datos continuos a low mid y high (según aplique)
    * Previous Qualification
    * Admission Grade
    * Curricular units 1st sem (grade) - (no dice continuous en la pagina pero float)
    * Curricular units 2nd sem (grade) -    "
    * Unemployment rate
    * Inflation rate
    * GDP


In [50]:
from sklearn.preprocessing import KBinsDiscretizer

In [86]:
#Defino Entrenamiento y testeo
X = df.drop("Target", axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , stratify=y, random_state=42)
#Separo Entrenamiento en entrenamiento y validación
train = X_train.join(y_train)
y = train["Target"]
training, validation =  train_test_split(train, test_size=0.2 , stratify=y, random_state=42)

In [ ]:
#Discretizar las columnas continuas

col_cont = X.select_dtypes(include=['float']).columns

discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

# Fit the discretizer on the selected columns
discretizer.fit(X[col_cont])

# Transform the selected columns
discretized_data_Train = discretizer.transform(X[col_cont])

discretized_data_Test = discretizer.transform([col_cont])

discretized_data = discretized_data.astype(int)

df[col_cont] = discretized_data

#### ID3

In [13]:
def entropia(df, objetivo, resultados):
    entropia = [0] * len(resultados)
    for i, val in enumerate(resultados):
        ci = df[df[objetivo]==val].shape[0]
        tot = df.shape[0]
        entropia[i]= -ci/tot * math.log(ci/tot) if not (ci==0) else 0
    return sum(entropia)

In [14]:
def entropiasbcjto(df, atributo, objetivo): #df datos, atributo que separa, columna objetivo
    valores = df[atributo].unique() #valores posibles del atributo
    aux = [0]*len(valores)
    s = df.shape[0] # cantidad de datos
    for i,valor in enumerate(valores):
        sv = df[df[atributo]==valor].shape[0] #cantidad de datos con atributo = valor
        EntrSv = entropia(df[df[atributo]==valor], objetivo, df[objetivo].unique())#Entropía de cada valor posible del atributo
        aux[i]=sv/s*EntrSv
    return sum(aux)

In [16]:
# Information Gain
def info_gain(df, atributo, objetivo):
    return entropia(df, objetivo, df[objetivo].unique()) - entropiasbcjto(df, atributo, objetivo)

In [18]:
def max_info (df, objetivo):
    atributos = df.drop(objetivo,axis = 1).columns
    max_info = -1
    max_info_atrib = None

    for col in atributos:
        gain_col = info_gain(df, col, objetivo)

        if gain_col > max_info:
            max_info = gain_col
            max_info_atrib = col
    
    return max_info_atrib



In [23]:
def generar_subarbol(atributo, data, objetivo, min_gain_split, min_sample_split):
    resultados = data[objetivo].unique() # Etiquetas resultado
    valores = data[atributo].unique() # valores del atributo
    tree = {} #subarbol

    for valor in valores:
        datos_valor = data[data[atributo]==valor] # me quedo con los datos de esta rama
        sig_nodo = max_info(datos_valor,objetivo)
        cant_datos = datos_valor.shape[0]
        info = info_gain(datos_valor, sig_nodo, objetivo) > min_gain_split #umbral información
        cant = cant_datos > min_sample_split # umbral cantidad de datos
        if info and cant: #si supera los dos umbrales
            tree[valor]="?" #marco la rama a extender
    
        else: #Si la rama no supera los umbrales hoja
            cant0 = len(datos_valor[datos_valor[objetivo]==resultados[0]])#cantidad de datos de cada etiqueta
            cant1 = len(datos_valor[datos_valor[objetivo]==resultados[1]])
            tree[valor] = resultados[0] if cant0>cant1 else resultados[1] #asigno la etiqueta mayoritaria
            data = data[data[atributo]!=valor] #Como es hoja saco estos datos
         
    return tree, data

In [24]:
def generar_arbol(raiz, val_prev, data, objetivo, min_gain_split, min_sample_split):
    if data.shape[0]!=0: 
        sig_nodo = max_info(data,objetivo)
        arbol, data = generar_subarbol(sig_nodo, data, objetivo, min_gain_split, min_sample_split)

        if val_prev != None :
            raiz[val_prev]={}
            raiz[val_prev][sig_nodo] = arbol
            nueva_raiz = raiz[val_prev][sig_nodo]

        if val_prev == None: # si es el comienzo del arbol
            raiz[sig_nodo]=arbol # raiz del arbol
            nueva_raiz = raiz[sig_nodo]

        for nodo, rama in list(nueva_raiz.items()):
            if rama =="?":
                generar_arbol(nueva_raiz, nodo, data[data[sig_nodo]==nodo], objetivo, min_gain_split, min_sample_split) 
                


In [26]:
def id3(data, objetivo, min_split_gain, min_sample_split):
    arbol={}
    generar_arbol(arbol, None, data, objetivo, min_split_gain, min_sample_split)
    return arbol

In [85]:
arbol = id3(training, "Target", 0.2, 10)

In [28]:
arbol

{'Curricular units 2nd sem (approved)': {0: 'Dropout',
  6: 'Enrolled',
  5: 'Enrolled',
  8: 'Enrolled',
  2: 'Dropout',
  7: 'Enrolled',
  4: 'Enrolled',
  1: 'Dropout',
  3: 'Enrolled',
  10: {'Age at enrollment': {27: 'Dropout',
    24: 'Dropout',
    20: {'Application mode': {1: 'Enrolled', 42: 'Dropout'}},
    42: 'Enrolled',
    28: {'Course': {9991: 'Enrolled', 9003: 'Dropout'}},
    46: 'Enrolled',
    19: {"Mother's qualification": {1: 'Enrolled',
      4: 'Enrolled',
      19: 'Enrolled',
      3: 'Dropout'}},
    37: 'Enrolled',
    30: 'Enrolled',
    45: 'Enrolled',
    23: 'Enrolled',
    25: 'Dropout',
    21: {'Course': {9070: 'Dropout',
      9238: 'Enrolled',
      9147: 'Enrolled',
      8014: 'Enrolled'}},
    22: 'Enrolled',
    32: {'Course': {9003: 'Dropout', 9991: 'Enrolled', 8014: 'Enrolled'}},
    29: 'Enrolled',
    54: 'Dropout',
    39: 'Enrolled',
    41: 'Enrolled',
    26: 'Enrolled'}},
  13: 'Enrolled',
  11: {'Curricular units 1st sem (enrolled)': {10

In [29]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [30]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [31]:
evaluate(arbol, df, "Target")

0.838381555153707

## Comparación con Scikit learn

In [70]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


In [39]:
df.dtypes

Marital status                                       int64
Application mode                                     int64
Application order                                    int64
Course                                               int64
Daytime/evening attendance\t                         int64
Previous qualification                               int64
Previous qualification (grade)                    category
Nacionality                                          int64
Mother's qualification                               int64
Father's qualification                               int64
Mother's occupation                                  int64
Father's occupation                                  int64
Admission grade                                   category
Displaced                                            int64
Educational special needs                            int64
Debtor                                               int64
Tuition fees up to date                              int

In [74]:
tree = DecisionTreeClassifier(criterion="entropy")

In [77]:
len(y_pred)

885

In [78]:
tree.fit(X_train, y_train)

y_pred = tree.predict(X_train)

print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

     Dropout       1.00      1.00      1.00      1137
    Enrolled       1.00      1.00      1.00      2402

    accuracy                           1.00      3539
   macro avg       1.00      1.00      1.00      3539
weighted avg       1.00      1.00      1.00      3539

